In [258]:
import json
import requests
from dotenv import load_dotenv
import os
import openai

load_dotenv()

True

In [259]:
openai.api_key = os.getenv('OPENAI_API_KEY')

HUE_IP = os.getenv('HUE_IP')
HUE_USERNAME = os.getenv('HUE_USERNAME')
GROUP_TO_CONTROL = '2'

KASA_IP = os.getenv('KASA_IP')

In [260]:
def get_context():
    r = requests.get(f'http://{HUE_IP}/api/{HUE_USERNAME}/groups')
    return r.json()

In [261]:
def make_prompt(context, command):
    framing = f'You are an AI that controls a smart home.'
    context = f'Here is the state of the devices in the home, in JSON format: {json.dumps(context)}'
    command = f'The user issues the command: {command}. Change the device state as appropriate.'
    formatting = f'Provide your response in JSON format.'

    return f'{framing} {context} {command} {formatting}'

def query_chatgpt(prompt):
    response = openai.Completion.create(
      model='text-davinci-003',
      prompt=prompt,
      temperature=0.7,
      max_tokens=2000,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0
    )
    return response

In [ ]:
context = {'lights': get_context()[GROUP_TO_CONTROL]['action']}
context['stereo'] = { 'state': 'off' }
context

In [ ]:
response = query_chatgpt(make_prompt(context, 'Set up for a party'))

In [ ]:
json_response = json.loads(response['choices'][0]['text'])
json_response

In [ ]:
# call the philips hue API with the new device state
r = requests.put(f'http://{HUE_IP}/api/{HUE_USERNAME}/groups/{GROUP_TO_CONTROL}/action', json=json_response['lights'])
r.json()
!kasa --host {KASA_IP} {json_response['stereo']['state']}